Imports

In [1]:
#imports
from tqdm import tqdm
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, Trainer, TrainingArguments
from utils import load_data, prepare_all_samples, get_entity_date_pairs, calculate_metrics
from bert_training import add_special_tokens, make_training_pairs, split_train_val, balance_classes, gold_lookup
from bert_extractor import get_context_window, mark_entity, mark_date, preprocess_input, bert_extraction

Data Loading

In [2]:
# Load data
df = load_data("data/synthetic.csv")
print(f"Loaded {len(df)} records")
#df

Loaded 101 records


In [3]:
# Prepare all samples
samples = prepare_all_samples(df)
print(f"Prepared {len(samples)} samples")
#samples[0]

Prepared 101 samples


BERT Base

In [4]:
# Load local model and tokenizer
#model_path = "./bert_model_training/base_model"
model_path = "google/bert_uncased_L-2_H-128_A-2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2, ignore_mismatched_sizes=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
#Example note
note_text = (
    "Patient diagnosed with asthma on 2024-08-02. "
    "Diabetes was ruled out on 2024-08-02. "
    "Family history of hypertension, last reviewed in 2022. "
    "Patient may have pneumonia, last seen on 2024-08-02."
)

# Example entity and date spans (positions are for illustration)
entity = {'start': 23, 'end': 29, 'label': 'asthma'}  # "asthma"
date = {'start': 33, 'end': 43, 'parsed': '2024-08-02'}  # "2024-08-02"

note_text, entity, date

('Patient diagnosed with asthma on 2024-08-02. Diabetes was ruled out on 2024-08-02. Family history of hypertension, last reviewed in 2022. Patient may have pneumonia, last seen on 2024-08-02.',
 {'start': 23, 'end': 29, 'label': 'asthma'},
 {'start': 33, 'end': 43, 'parsed': '2024-08-02'})

In [6]:
#Test context window
context = get_context_window(note_text, entity['start'], date['start'], window_size=50)
print("Context window:\n", context)

Context window:
 Patient diagnosed with asthma on 2024-08-02. Diabetes was ruled out on 2024-08-02. 


In [7]:
#Test entity marking
entity_text = note_text[entity['start']:entity['end']]
offset = context.find(entity_text)
entity_rel = {'start': offset, 'end': offset + len(entity_text)}

marked_entity = mark_entity(context, entity_rel)
print("Entity marked:\n", marked_entity)

Entity marked:
 Patient diagnosed with [E]asthma[E] on 2024-08-02. Diabetes was ruled out on 2024-08-02. 


In [8]:
#Test date marking
date_text = note_text[date['start']:date['end']]
offset_date = context.find(date_text)
date_rel = {'start': offset_date, 'end': offset_date + len(date_text)}

marked_date = mark_date(context, date_rel)
print("Date marked:\n", marked_date)

Date marked:
 Patient diagnosed with asthma on [D]2024-08-02[D]. Diabetes was ruled out on 2024-08-02. 


The below cell needs fixing!

In [9]:
#Do full pre-processing
preprocessed = preprocess_input(note_text, entity, date, window_size=20)
print("Preprocessed input:\n", preprocessed)

Preprocessed input:
 ient diagnosed with [E]asthma[[D]E] on 2024[D]-08-02. Diabetes


In [10]:
# Process samples
predictions = []

#for sample in tqdm(samples, desc="Samples"):
    #pairs = get_entity_date_pairs(sample['entities_list'], sample['dates'])
    #for pair in pairs:
        #text = f"Does '{pair['entity_label']}' relate to '{pair['date']}'? {sample['note_text']}..."
        #pred, conf = bert_extraction(text, model, tokenizer)
        #if pred == 1:
            #predictions.append({'entity_label': pair['entity_label'], 'date': pair['date'], 'confidence': conf})

for sample in tqdm(samples, desc="Samples"):
    pairs = get_entity_date_pairs(sample['entities_list'], sample['dates'])
    for pair in pairs:
        entity = pair['entity']
        date = pair['date_info']
        pred, conf = bert_extraction(sample['note_text'], entity, date, model, tokenizer, window_size=100)
        if pred == 1:
            predictions.append({'entity_label': entity['label'], 'date': date['parsed'], 'confidence': conf})

Samples: 100%|██████████| 101/101 [00:12<00:00,  7.98it/s]


In [11]:
#Look at predictions
predictions

[]

In [12]:
# Calculate metrics
metrics = calculate_metrics(predictions, df)
metrics

{'precision': 0, 'recall': 0.0, 'f1': 0, 'tp': 0, 'fp': 0, 'fn': 214}

BERT Finetuning

In [13]:
# 1. Add special tokens to tokenizer (do this before model.resize_token_embeddings)

# Add special tokens to tokenizer
add_special_tokens(tokenizer)

# Resize model embeddings to match new tokenizer size
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(30524, 128, padding_idx=0)

In [14]:
# 2. Prepare training pairs
training_pairs = make_training_pairs(samples, gold_lookup, window_size=100)

In [15]:
# 3. (Optional) Balance classes for better F1
training_pairs = balance_classes(training_pairs, ratio=1.0)  # 1:1 pos:neg

In [16]:
# 4. Split into train/val
train_pairs, val_pairs = split_train_val(training_pairs, val_frac=0.2)

In [17]:
# 5. Convert to HuggingFace Datasets
train_dataset = Dataset.from_pandas(train_pairs)
val_dataset = Dataset.from_pandas(val_pairs)

In [18]:
# 6. Tokenization function
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

In [19]:
#Tokenization
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/278 [00:00<?, ? examples/s]

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

In [20]:
#Look at tokenized datasets
tokenized_train

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 278
})

In [21]:
#Define training args
training_args = TrainingArguments(
    output_dir="./bert_model_training/bert_finetuned",
    per_device_train_batch_size=16,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    save_total_limit=1,
    load_best_model_at_end=True,
)

In [22]:
#Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

In [23]:
#Train
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.699400,0.696198
2,0.695400,0.695089
3,0.682200,0.693431
4,0.679000,0.692790
5,0.681200,0.690405
6,0.664700,0.689230
7,0.646600,0.686603
8,0.641600,0.685895
9,0.642900,0.684482
10,0.649900,0.684691


TrainOutput(global_step=180, training_loss=0.6645816564559937, metrics={'train_runtime': 44.3532, 'train_samples_per_second': 62.679, 'train_steps_per_second': 4.058, 'total_flos': 882988492800.0, 'train_loss': 0.6645816564559937, 'epoch': 10.0})

In [24]:
#Process samples
predictions = []

for sample in tqdm(samples, desc="Samples"):
    pairs = get_entity_date_pairs(sample['entities_list'], sample['dates'])
    for pair in pairs:
        entity = pair['entity']
        date = pair['date_info']
        pred, conf = bert_extraction(sample['note_text'], entity, date, model, tokenizer, window_size=100)
        if pred == 1:
            predictions.append({'entity_label': entity['label'], 'date': date['parsed'], 'confidence': conf})

Samples: 100%|██████████| 101/101 [00:10<00:00, 10.02it/s]


In [25]:
#Look at predictions
predictions

[{'entity_label': 'rheumatoid_arthritis',
  'date': '2024-10-23',
  'confidence': 0.5512459874153137},
 {'entity_label': 'pneumonia',
  'date': '2024-10-23',
  'confidence': 0.5362464785575867},
 {'entity_label': 'gerd',
  'date': '2024-10-23',
  'confidence': 0.5325896143913269},
 {'entity_label': 'meningitis',
  'date': '2024-10-23',
  'confidence': 0.6296129822731018},
 {'entity_label': 'multiple_sclerosis',
  'date': '2024-09-27',
  'confidence': 0.5492989420890808},
 {'entity_label': 'bronchitis',
  'date': '2025-01-29',
  'confidence': 0.5313732028007507},
 {'entity_label': 'tension_headache',
  'date': '2025-01-29',
  'confidence': 0.5267938375473022},
 {'entity_label': 'gerd',
  'date': '2025-01-29',
  'confidence': 0.5116325616836548},
 {'entity_label': 'congenital malformation',
  'date': '2024-09-27',
  'confidence': 0.5530809760093689},
 {'entity_label': 'copd',
  'date': '2025-05-07',
  'confidence': 0.5368807315826416},
 {'entity_label': 'hyperlipidemia',
  'date': '2025-

In [26]:
# Calculate metrics
metrics = calculate_metrics(predictions, df)
metrics

{'precision': 0.2391304347826087,
 'recall': 0.616822429906542,
 'f1': 0.3446475195822454,
 'tp': 132,
 'fp': 420,
 'fn': 82}